# Borrower reliability research

The customer is the credit department of the bank. It is necessary to find out whether the marital status and the number of children of the client affect the fact of repaying the loan on time. Input data from the bank - statistics on the solvency of customers.

The results of the study will be taken into account when building a **credit scoring** model - a special system that evaluates the ability of a potential borrower to repay a loan to a bank.

## Step 1. Open the data file and examine the general information

In [1]:
import pandas as pd
import numpy as np

Reading the data:

In [6]:
df = pd.read_csv('data.csv')

Let's look at the first 5 rows of the array for reference:

In [7]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


The table has 12 columns. Data type — `int64`, `float64`, `object`

According to the data documentation:
* `children` - the number of children in the family;
* `days_employed` - work experience in days;
* `dob_years` - client's age in years;
* `education` - education of the client;
* `education_id` - education identifier;
* `family_status` - marital status;
* `family_status_id` - marital status identifier;
* `gender` - gender of the client;
* `income_type` - employment type;
* `debt` - whether he had a debt to repay loans;
* `total_income` - income per month;
* `purpose` - the purpose of the loan;

Note that there are **sign - ID** pairs in the columns (`education - education_id`, `family_status - family_status_id`), this can be useful for checking the correctness of filling in the cells in the corresponding columns.

Let's get general information about the data in the df table:

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [9]:
df[df['days_employed'].isna()]['total_income'].unique()

array([nan])

In [10]:
df[df['total_income'].isna()]['days_employed'].unique()

array([nan])

* **The number of values in the columns varies. This means there are missing values in the data (columns with `days_employed` and `total_income` quantitative values).**
* **Wherever there is a `days_employed` gap there is a `total_income` gap and vice versa**
* **Data types in columns `days_employed` and `total_income`, current type `real`, but more logical - `integer`**

Let's check how the columns of the attribute-ID pair correlate (I wrote about them earlier):

In [11]:
df.groupby(['education_id','education'])[['children']].count()

children
education_id education                    
0            ВЫСШЕЕ                    274
             Высшее                    268
             высшее                   4718
1            СРЕДНЕЕ                   772
             Среднее                   711
             среднее                 13750
2            НЕОКОНЧЕННОЕ ВЫСШЕЕ        29
             Неоконченное высшее        47
             неоконченное высшее       668
3            НАЧАЛЬНОЕ                  17
             Начальное                  15
             начальное                 250
4            УЧЕНАЯ СТЕПЕНЬ              1
             Ученая степень              1
             ученая степень              4

* **`education` - we see an error in filling in categorical variables, different register - this needs to be corrected**

In [12]:
df.groupby(['family_status_id','family_status'])[['children']].count()

,,children
family_status_id,family_status,
0,женат / замужем,12380
1,гражданский брак,4177
2,вдовец / вдова,960
3,в разводе,1195
4,Не женат / не замужем,2813


* **`family_status` - less critical here, but the case also needs to be greeted to the same format**

To complete the review of categorical columns, check the unique values for `gender` and `income_type`:

In [13]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [14]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

- **`XNA` - I guess it's NA and can be replaced with an empty string, for convenience. We will not be able to restore the value***

Let's continue the review and see what happens with the numeric columns:

In [15]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


This is also not without problems:
- **`children` minimum value `-1`, maximum `20` look like typos when entering data**
- **`dob_years` minimum value `0` (we will not be able to restore the value)**
- **`days_employed` there are both negative values and extremely large ones (`experience >> 100 years`)**

Let's also see if there are obvious duplicates in the selection (even in its current state, with all the inaccuracies that I described above):

In [16]:
df.duplicated().sum()

54

**Conclusion**

Each row of the table contains data about the borrower of the bank. Some of the columns describe the socio-demographic characteristics of the client: gender, age, work experience, education data, marital status and number of children. The rest are economic (including risky) metrics for the client: the presence of delays, the amount of monthly income, the purpose of obtaining a loan.

Additionally, it should be noted that:
- there are omissions
- part of the columns must be converted to an integer data type
- quantitative variables are either extremely high or negative values
- there are typos in categorical variables
- there are obvious duplicates

To move forward, you need to fix problems in the data.

## Step 2. Data preprocessing

### Pass processing

The next task of the project is to fill in the blanks.
In data review, I found gap issues in two columns `days_employed` and `total_income`

- In both cases, gaps in *quantitative variables*.
- Also, gaps in both columns are random, and wherever there is a gap `days_employed` there is a gap `total_income` and vice versa
- Gaps in such variables are filled with *characteristic values*.

**Possible reason for gaps**
- The bank does not always know everything about the salary and seniority of the client, for example, if this is an incoming application from a new client who has never been served by this bank before. At the same time, the client can leave an application on the site, and the bank must decide on the issuance based on the data indicated in the questionnaire (in our case, information about the length of service and income level was not filled out)
- There may also be typos when entering data
- Or technical errors, like with the `days_employed` field


**Note**
If we replace the gaps throughout the array with the arithmetic mean, the result will be as inaccurate as possible, so I suggest splitting the array into more stable subgroups. Since clustering has not yet been read to us, I will assume that a strong influence on `length of service` and `income level` can have:


- type of employment
- type of education
- gender
- age

At the same time, the last parameter is given to us in years, without breaking down into groups, so in order to more accurately fill in the gaps, we will have to categorize by age right now.

In [17]:
def age_group(age):
    if age <= 18:
        return '0-18'
    elif  age >= 19 and age <= 25:
        return '19-25'
    elif  age >= 26 and age <= 35:
        return '26-35'
    elif  age >= 36 and age <= 55:
        return '36-55'
    else:
        return '56+'

In [18]:
df['age_id'] = df['dob_years'].apply(age_group)

In [19]:
## verification
df.groupby(['age_id'])[['education_id']].count()

,education_id
age_id,
0-18,101
19-25,1234
26-35,5360
36-55,10461
56+,4369


Let's also fix the case for text columns:

In [20]:
for i in df:
    try:
        df[i] = df[i].str.lower()
    except:
        df[i] = df[i]

In [21]:
## verification
df.groupby(['education_id','education'])[['children']].count()

,,children
education_id,education,
0,высшее,5260
1,среднее,15233
2,неоконченное высшее,744
3,начальное,282
4,ученая степень,6


In [22]:
## verification
df.groupby(['family_status_id','family_status'])[['children']].count()

,,children
family_status_id,family_status,
0,женат / замужем,12380
1,гражданский брак,4177
2,вдовец / вдова,960
3,в разводе,1195
4,не женат / не замужем,2813


Let's fix the values in the `children` column, under the suspicion of -1 and 20

In [23]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [24]:
len(df[(df['children'] == 20) | (df['children'] == -1)])

123

I'm assuming these are all typos.

In [25]:
df['children'] = df['children'].replace(20,2)
df['children'] = df['children'].replace(-1,1)

Change the **XNA** value to **None** in the `gender` column

In [26]:
df['gender'] = df['gender'].replace('xna','none')

In [27]:
df['gender'].unique()

array(['f', 'm', 'none'], dtype=object)

**After basic manipulations, let's move on to filling in the gaps**
**days_employed**

Above, I wrote about extreme values in it, as well as the presence of both positive and negative variables.
This suggests that the problem is deeper and it is not possible to simply fill in the gaps with column averages.
Let's look at the distribution of values in the days_employed column for values > 0:

In [28]:
df[df['days_employed'] > 0]['days_employed'].describe()

count      3445.000000
mean     365004.309916
std       21075.016396
min      328728.720605
25%      346639.413916
50%      365213.306266
75%      383246.444219
max      401755.400475
Name: days_employed, dtype: float64

It can be seen that even the minimum value of experience here is > 900 years
Now let's look at the distributions for seniority values < 0:

In [29]:
df[df['days_employed'] < 0]['days_employed'].describe()

count    15906.000000
mean     -2353.015932
std       2304.243851
min     -18388.949901
25%      -3157.480084
50%      -1630.019381
75%       -756.371964
max        -24.141633
Name: days_employed, dtype: float64

Here everything looks more believable (assuming that negative values are actually positive).

Let's check the maximum value of experience.

In [30]:
df[df['days_employed'] == df[df['days_employed'] < 0]['days_employed'].min()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_id
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,f,сотрудник,0,186178.934089,операции с недвижимостью,56+


The age of the client bank is 61 years, while the experience is ~50 years.

At the same time, the 25% quantile for the negative values of the days_employed column is 3157 days (or 8.6 years), which means that 75% of records have an experience greater than -3157 (that is, 75% of customers have an experience of less than 8.6 years).

**Suggestions for adjusting the `days_employed` column:**
- replace positive seniority values with an empty string
- take the remaining negative values modulo
- fill in the gaps in the column with the average values for the groups `type of employment-type of education-gender-age-marital status`

**Replacing positive experience values with an empty string**

In [31]:
for i in range(len(df)):
    if df.loc[i, 'days_employed'] > 0:
        df.loc[i, 'days_employed'] = np.nan

In [32]:
### verification
df[df['days_employed'] > 0]['days_employed'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: days_employed, dtype: float64

**We take the remaining negative values modulo**

In [33]:
df['days_employed'] = df['days_employed'].abs()

In [34]:
### verification
df['days_employed'].min()

24.14163324048118

**Search for characteristic values of experience for subgroups**

In [35]:
avg_d_empl = df.groupby(['age_id','family_status_id','income_type','education_id', 'gender'])\
      .agg({'days_employed': ['count', 'sum']}).reset_index()

In [36]:
avg_d_empl.columns =  avg_d_empl.columns.droplevel(level = 1)

In [37]:
avg_d_empl.columns = ['age_id', 'family_status_id', 'income_type', 'education_id', 'gender','count', 'sum']

In [38]:
len(avg_d_empl)

461

We got a large reference table with average values, we will check for the worst condition:
     when there were examples in the subgroup, but not a single value of experience was found.

In [39]:
avg_d_empl[(avg_d_empl['count'] != 0) & (avg_d_empl['sum'] ==  0)].head(5)

,age_id,family_status_id,income_type,education_id,gender,count,sum


I added too many groups for the directory, zero values ​​appeared - such a directory will not suit us.

In [40]:
avg_d_empl = df.groupby(['gender', 'age_id'])\
      .agg({'days_employed': ['count', 'sum']}).reset_index()

In [41]:
avg_d_empl.columns =  avg_d_empl.columns.droplevel(level = 1)

In [42]:
avg_d_empl.columns = ['gender','age_id','count', 'sum']

In [43]:
avg_d_empl[(avg_d_empl['count'] != 0) & (avg_d_empl['sum'] ==  0)].head(5)

,gender,age_id,count,sum


In [44]:
len(avg_d_empl)

11

In [45]:
avg_d_empl.head(5)

,gender,age_id,count,sum
0,f,0-18,48,1.219516e+05
1,f,19-25,636,5.901306e+05
2,f,26-35,2836,4.865908e+06
3,f,36-55,5619,1.628293e+07
4,f,56+,806,3.192357e+06


There are no zero values of the sums, so the table can be used as a reference in this form.

**We pull the values from the reference book into our main table**

In [46]:
df = df.merge(avg_d_empl, on=['age_id','gender'], how='left')

In [47]:
df['days_employed'] = np.where(df['days_employed'].isnull(), df['sum'], df['days_employed'])

In [48]:
df.drop(['sum', 'count'], axis=1, inplace=True)

In [49]:
### verification
df['days_employed'].describe()

count    2.152500e+04
mean     1.716391e+06
std      4.095927e+06
min      2.414163e+01
25%      1.025608e+03
50%      2.609841e+03
75%      1.421790e+06
max      1.628293e+07
Name: days_employed, dtype: float64

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
 12  age_id            21525 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB


**Result**
Replaced gaps in `days_employed`

**Fill in the gaps in the `total_income` column in the same way**
The gap situation is simpler here than in the seniority column, since all values in the column are positive.
My task will be:
- to the search for characteristic values for subgroups
- replacement of gaps with characteristic values

In [51]:
df['total_income'].describe()

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [52]:
avg_inc = df.groupby(['gender', 'age_id'])\
      .agg({'total_income': ['count', 'sum']}).reset_index()

In [53]:
avg_inc.columns =  avg_inc.columns.droplevel(level = 1)

In [54]:
avg_inc.columns = ['gender','age_id','count', 'sum']

In [55]:
avg_inc[(avg_inc['count'] != 0) & (avg_inc['sum'] ==  0)].head(5)

,gender,age_id,count,sum


In [56]:
len(avg_inc)

11

**We pull the values from the reference book into our main table**

In [57]:
df = df.merge(avg_inc, on=['age_id','gender'], how='left')
df['total_income'] = np.where(df['total_income'].isnull(), df['sum'], df['total_income'])
df.drop(['sum', 'count'], axis=1, inplace=True)

In [58]:
### verification
df['total_income'].describe()

count    2.152500e+04
mean     6.250003e+07
std      2.097614e+08
min      2.066726e+04
25%      1.077982e+05
50%      1.564003e+05
75%      2.377920e+05
max      1.003516e+09
Name: total_income, dtype: float64

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  age_id            21525 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB


**Result**
Replaced gaps in `total_income`

**Conclusion**

- At this point, in addition to the gaps, I took the liberty of correcting all other shortcomings in the data set
- I did not use the `.fillna()` method to replace gaps, because for this purpose I made a reference book with characteristic values ​​​​for subgroups (I went the more complicated way)

In data review, I found gap issues in two columns `days_employed` and `total_income`

- In both cases, gaps in *quantitative variables*.
- Also, gaps in both columns are random, and wherever there is a gap `days_employed` there is a gap `total_income` and vice versa
- Gaps in such variables are filled with *characteristic values*.

**Possible reason for gaps**
- The bank does not always know everything about the salary and seniority of the client, for example, if this is an incoming application from a new client who has never been served by this bank before. At the same time, the client can leave an application on the site, and the bank must decide on the issuance based on the data indicated in the questionnaire (in our case, information about the length of service and income level was not filled out)
- There may also be typos when entering data
- Or technical errors, like with the `days_employed` field

### Replacing the data type

Even at the data review stage, I found:
- Data types in `days_employed` and `total_income` columns, current type is real, but more logical - integer

We removed the gaps in the last paragraph, so there should be no problems.

In [60]:
cols = ['days_employed', 'total_income']
df[cols] = df[cols].astype(int)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
 12  age_id            21525 non-null  object
dtypes: int64(7), object(6)
memory usage: 2.3+ MB


**Conclusion**

This point of the study turned out to be somewhat small.
Let's answer the main questions:
- for columns days_employed and total_income the data type was changed from real to integer
- the `astype()` translation method was used, we know from the lectures that when using the `to_numeric()` method, all numbers will have the *float* data type, and in this case we need *int* and the `astype method ()` allows you to specify it.

### Handling duplicates

After all the data manipulation and corrections, we can finally look for duplicates.
Obvious duplicates:

In [62]:
### data array before deleting duplicates
len(df)

21525

In [63]:
df.duplicated().sum()

71

In [64]:
df = df.drop_duplicates()
df = df.dropna().reset_index(drop = True)

In [65]:
## verification
df.duplicated().sum()

0

In [66]:
len(df)

21454

**Conclusion**

- Obvious duplicates have been removed in this item of the study
- By condition, we do not know the unique client ID, so we can dedupe by those columns that are given to us in the task, while we have already corrected all possible inaccuracies and typos earlier, which is why I applied the `drop_duplicates()` method to the entire array without selecting groups columns
- The reason for the appearance of duplicates in our task, in addition to a technical failure, may be the repeated sending of an application by the same client with the same data

### Lemmatization

Let's lemmitize the column containing the description of the purpose of the loan - `purpose`.      
**Lemmatization** - bringing a word to its dictionary form (lemma).

Recording forms in the dictionary of Russian words:
- for nouns - nominative case, singular
- for adjectives - nominative case, singular, masculine
- for verbs, participles, gerunds - a verb in the infinitive of the imperfect form.

In [65]:
from pymystem3 import Mystem
m = Mystem()

My idea is to collect a large list of all lemmas throughout the array and understand the most frequently occurring words in order to understand the purpose of the loan. 

In [66]:
lemmas = []
for i in range(len(df)):
    try:
        lemmas +=  m.lemmatize(df.loc[i, 'purpose'])
    except:
        lemmas += ''

In [67]:
from collections import Counter

In [68]:
print(Counter(lemmas)) 

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


**Conclusion**

- conducted **lemmitization** (bringing a word to its dictionary form) of texts in the `loan purpose` column
- the main purposes of acquiring a loan include:
     - real estate and housing (operations or construction)
     - automobile
     - education
     - wedding
- the resulting representation will help for further research (categorization and study of data dependencies)

### Data categorization

**Categorization** - combining data into categories based on a common feature.   
Previously, I already performed this operation with `client age`, now you need to do the same for `income` and `loan purpose`.  

This will make it easier to work with the data and make it easier to find patterns in them.

In [69]:
def inc_group(income):
    if income <= 100000:
        return '0-100'
    elif  income > 100000 and income <= 150000:
        return '100-150'
    elif  income > 150000 and income <= 250000:
        return '150-250'
    elif  income > 250000:
        return '250+'
    else:
        return 'other'

In [70]:
df['inc_group'] = df['total_income'].apply(inc_group)
### verification
df.groupby(['inc_group'])[['education_id']].count()

,education_id
inc_group,
0-100,4463
100-150,5704
150-250,6372
250+,4915


In the **Lemmitization** section, we have identified the most popular loan purposes, it will be convenient to create a `purpose_type` category column in our dataset.

In [71]:
for i in range(len(df)):
    try:
        if 'недвижимость' in m.lemmatize(df.loc[i, 'purpose']) or 'жилье' in m.lemmatize(df.loc[i, 'purpose']):
            df.loc[i,'purpose_type'] = 'недвижимость'
        elif 'автомобиль' in m.lemmatize(df.loc[i, 'purpose']):
            df.loc[i,'purpose_type'] = 'автомобиль'
        elif 'образование' in m.lemmatize(df.loc[i, 'purpose']):
            df.loc[i,'purpose_type'] = 'образование'
        elif 'свадьба' in m.lemmatize(df.loc[i, 'purpose']):
            df.loc[i,'purpose_type'] = 'свадьба'
        else:
            df.loc[i,'purpose_type'] = 'прочее'
    except:
        df.loc[i,'purpose_type'] = ''

In [72]:
df['purpose_type'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_type, dtype: int64

In [73]:
df['age_id'].value_counts()

36-55    10434
26-35     5351
56+       4336
19-25     1232
0-18       101
Name: age_id, dtype: int64

**Conclusion**

- the data was categorized by the columns `age of the client` and `loan purpose`
- the most popular loan purpose is *real estate* (acquisition or construction)
- the main age category of borrowers - *36-55 years*

## Step 3: Answer the questions

- Is there a relationship between marital status and loan repayment on time?

In [74]:
pivot_chld = df.pivot_table(
      values=['age_id'], index='children', columns='debt',fill_value=0, aggfunc='count')

In [75]:
### Added a line total
pivot_chld['total'] = pivot_chld['age_id'][0] + pivot_chld['age_id'][1]

In [76]:
### Calculate the share of the total
pivot_chld['0_%%'] = round(pivot_chld['age_id'][0]*100/pivot_chld['total'], 1)
pivot_chld['1_%%'] = round(pivot_chld['age_id'][1]*100/pivot_chld['total'], 1)

In [77]:
pivot_chld

age_id        total   0_%% 1_%%
debt          0     1                   
children                                
0         13028  1063  14091   92.5  7.5
1          4410   445   4855   90.8  9.2
2          1926   202   2128   90.5  9.5
3           303    27    330   91.8  8.2
4            37     4     41   90.2  9.8
5             9     0      9  100.0  0.0

**Conclusion**

Such a dependence is indeed traced:
- for borrowers without children, the probability of default is 7.5%
- for borrowers with children, the probability of default is higher than 8.2%, while the probability of non-payment increases as the number of children grows (the exception is families with 3 and 5 children, but since the sample is small, %% metrics are very sensitive to small deviations in absolute numbers)

- Is there a relationship between the level of income and repayment of the loan on time?

In [78]:
pivot_inc = df.pivot_table(
      values=['age_id'], index='inc_group', columns='debt',fill_value=0, aggfunc='count')

In [79]:
### Added a line total
pivot_inc['total'] = pivot_inc['age_id'][0] + pivot_inc['age_id'][1]

In [80]:
### Calculate the share of the total
pivot_inc['0_%%'] = round(pivot_inc['age_id'][0]*100/pivot_inc['total'], 1)
pivot_inc['1_%%'] = round(pivot_inc['age_id'][1]*100/pivot_inc['total'], 1)

In [81]:
pivot_inc

age_id      total  0_%% 1_%%
debt           0    1                 
inc_group                             
0-100       4109  354  4463  92.1  7.9
100-150     5213  491  5704  91.4  8.6
150-250     5840  532  6372  91.7  8.3
250+        4551  364  4915  92.6  7.4

**Additionally**  
It is interesting how borrowers with an income of 0-100 are distributed by age groups.
Suddenly there are many very young borrowers who are simply not approved for loans due to their age.

In [82]:
pivot_inc_age = df.pivot_table(
      values=['children'], index=['inc_group','age_id'], columns='debt',fill_value=0, aggfunc='count')

In [83]:
pivot_inc_age.head(5)

children     
debt                    0    1
inc_group age_id              
0-100     0-18         22    1
          19-25       267   31
          26-35       855  101
          36-55      1791  158
          56+        1174   63

The hypothesis was not confirmed

**Conclusion**

- Borrowers with high income (more than 250 thousand rubles) fall into arrears less than all other categories of income
- At the same time, the higher the income, the less likely it is to go into delay
- Also, the percentage of delays is low in the category with income from 0 to 100 thousand rubles, this can be explained by the fact that banks approve smaller amounts for them and check them more carefully

- How do different purposes of the loan affect its repayment on time?

In [84]:
pivot_purp = df.pivot_table(
      values=['age_id'], index='purpose_type', columns='debt',fill_value=0, aggfunc='count')

In [85]:
### Added a line total
pivot_purp['total'] = pivot_purp['age_id'][0] + pivot_purp['age_id'][1]

In [86]:
### Calculate the share of the total
pivot_purp['0_%%'] = round(pivot_purp['age_id'][0]*100/pivot_purp['total'], 1)
pivot_purp['1_%%'] = round(pivot_purp['age_id'][1]*100/pivot_purp['total'], 1)

In [87]:
pivot_purp

age_id       total  0_%% 1_%%
debt              0    1                  
purpose_type                              
автомобиль     3903  403   4306  90.6  9.4
недвижимость  10029  782  10811  92.8  7.2
образование    3643  370   4013  90.8  9.2
свадьба        2138  186   2324  92.0  8.0

**Conclusion**

The objectives of lending also strongly share risk:
- the highest probability of non-repayment of the loan - a loan for the purchase of a car and for education
- the lowest risk - for real estate loans

- Is there a relationship between marital status and loan repayment on time?

In [88]:
pivot_fam = df.pivot_table(
      values=['age_id'], index='family_status', columns='debt',fill_value=0, aggfunc='count')

In [89]:
### Added a line total
pivot_fam['total'] = pivot_fam['age_id'][0] + pivot_fam['age_id'][1]

In [90]:
### Calculate the share of the total
pivot_fam['0_%%'] = round(pivot_fam['age_id'][0]*100/pivot_fam['total'], 1)
pivot_fam['1_%%'] = round(pivot_fam['age_id'][1]*100/pivot_fam['total'], 1)

In [92]:
pivot_fam

age_id       total  0_%% 1_%%
debt                       0    1                  
family_status                                      
в разводе               1110   85   1195  92.9  7.1
вдовец / вдова           896   63    959  93.4  6.6
гражданский брак        3763  388   4151  90.7  9.3
женат / замужем        11408  931  12339  92.5  7.5
не женат / не замужем   2536  274   2810  90.2  9.8

**Conclusion**

- the highest risk is concentrated in the category of unmarried / unmarried borrowers, as well as among people who are in a civil marriage
- at the same time, the lowest among official marriages, as well as people who are divorced, widows / widowers, but there is nothing strange here. The last two categories are the smallest and banks check "loners" more carefully (a similar example was with low risk on salary accruals of less than 100 thousand rubles.)

## Step 4. General conclusion

The project analyzed the bank's borrower base, it was necessary to analyze the client's solvency depending on a number of parameters (number of children, purpose of the loan, income level)

The project consisted of two major phases:
- data preprocessing and correction of errors and inaccuracies in it
- hypothesis testing

The first part was the most time consuming (I spent a lot of time on it) and included:
- filling in blanks
- correction of data types
- remove duplicates
- text lemmitization (field - purpose of the loan)
- categorization of a number of characteristics (age, purpose of the loan, income level)

The second part was devoted to the study of the dependence of solvency.
Within the framework of this paragraph, conclusions were drawn about the correlation of exit in arrears from:
- the number of children in the family (moreover, the more children, the higher the probability of loan default)
- income level (the data showed that the higher the income, the lower the probability of default, while there is an exception - in the group of borrowers with an income of less than 100 thousand rubles, there are slightly fewer defaults, since banks approve smaller amounts for such clients and check them more carefully before issuing)
- purposes of lending, it was possible to find out that loans for a car and education carry the greatest risk